<a href="https://colab.research.google.com/github/YashsTiwari/ML_DL_revision/blob/main/coding_AlexNet_using_Tf_and_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d ifigotin/imagenetmini-1000

Dataset URL: https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000
License(s): unknown
100% 3.92G/3.92G [01:36<00:00, 23.5MB/s]
100% 3.92G/3.92G [01:36<00:00, 43.6MB/s]


In [7]:
import zipfile
zip_ref=zipfile.ZipFile('/content/imagenetmini-1000.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [12]:
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/imagenet-mini/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(227,227)
)

val_ds=keras.utils.image_dataset_from_directory(
    directory='/content/imagenet-mini/val',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(227,227)
)

Found 34745 files belonging to 1000 classes.
Found 3923 files belonging to 1000 classes.


In [13]:
def process(image,label):
  image=tf.cast(image/255. ,tf.float32)
  return image,label

train_ds=train_ds.map(process)
val_ds=val_ds.map(process)

In [17]:
# AlexNet model

model=Sequential()

model.add(Conv2D(96, kernel_size=(11,11), strides=4, padding='valid', activation='relu', input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=2, padding='valid'))
model.add(Conv2D(256, strides=1, kernel_size=(5,5), activation='relu', padding='same'))
model.add(Conv2D(384, strides=1, kernel_size=(3,3), padding='same'))
model.add(Conv2D(384, strides=1, kernel_size=(3,3), padding='same'))
model.add(Conv2D(256, strides=1, kernel_size=(3,3), padding='same'))
model.add(MaxPooling2D(pool_size=(3,3), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 27, 27, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 27, 27, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 27, 27, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │   177,213,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,838,952 (769.95 MB)

 Trainable params: 201,838,952 (769.95 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 153s 128ms/step - accuracy: 0.0011 - loss: 6.9082 - val_accuracy: 7.6472e-04 - val_loss: 6.9014
Epoch 2/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 174s 114ms/step - accuracy: 0.0019 - loss: 6.8898 - val_accuracy: 0.0010 - val_loss: 6.8688
Epoch 3/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 133s 106ms/step - accuracy: 0.0033 - loss: 6.8430 - val_accuracy: 0.0023 - val_loss: 6.8242
Epoch 4/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 141s 105ms/step - accuracy: 0.0068 - loss: 6.7907 - val_accuracy: 0.0033 - val_loss: 6.7659
Epoch 5/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 116s 107ms/step - accuracy: 0.0080 - loss: 6.7200 - val_accuracy: 0.0064 - val_loss: 6.6979
Epoch 6/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 116s 107ms/step - accuracy: 0.0102 - loss: 6.6361 - val_accuracy: 0.0084 - val_loss: 6.6159
Epoch 7/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 141s 106ms/step - accuracy: 0.0126 - loss: 6.5226 - val_accuracy: 0.0105 - val_loss: 6.4773
Epoch 8/10
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 145s 108ms/step 

### This AlexNet model is run on a very small subset of the original dataset and run only for 10 epochs so the accuracy is not good.